In [5]:
# ==========================
# 1. Import Libraries
# ==========================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, classification_report, confusion_matrix
from xgboost import XGBRegressor, XGBClassifier

# ==========================
# 2. Load & Preprocess Data
# ==========================
df = pd.read_csv("../data/weatherHistory.csv")

# Parse datetime
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], errors='coerce', utc=True)
df['Formatted Date'] = df['Formatted Date'].dt.tz_localize(None)

# Extract datetime features
df['year'] = df['Formatted Date'].dt.year
df['month'] = df['Formatted Date'].dt.month
df['day'] = df['Formatted Date'].dt.day
df['hour'] = df['Formatted Date'].dt.hour

# Synthetic thunderstorm label
df['thunderstorm'] = np.where(
    (df['Humidity'] > 0.85) &
    (df['Wind Speed (km/h)'] > 25) &
    (df['Pressure (millibars)'] < 1005), 1, 0
)

# Drop unused columns
df = df.drop(columns=['Summary', 'Daily Summary', 'Loud Cover'], errors='ignore')

# Handle categorical column
if 'Precip Type' in df.columns:
    df['Precip Type'] = df['Precip Type'].fillna(df['Precip Type'].mode()[0])
    df = pd.get_dummies(df, columns=['Precip Type'], drop_first=True)

# ==========================
# 3. Train/Val/Test split
# ==========================
train_df = df[(df['year'] >= 2006) & (df['year'] <= 2012)].copy()
val_df   = df[(df['year'] > 2012) & (df['year'] <= 2014)].copy()
test_df  = df[(df['year'] > 2014) & (df['year'] <= 2016)].copy()

# ==========================
# 4. Target Engineering for multiple horizons
# ==========================
horizons = [1, 3, 6, 12, 24]

for h in horizons:
    for split in [train_df, val_df, test_df]:
        split[f'Wind_Speed_next_{h}h'] = split['Wind Speed (km/h)'].shift(-h)
        split[f'Thunderstorm_next_{h}h'] = split['thunderstorm'].shift(-h)

# Drop NaNs introduced by shifting
train_df = train_df.dropna().reset_index(drop=True)
val_df = val_df.dropna().reset_index(drop=True)
test_df = test_df.dropna().reset_index(drop=True)

# ==========================
# 5. Feature Engineering
# ==========================
def add_lag_features(df, lags=[1,3,6,12]):
    for lag in lags:
        df[f'Wind_Speed_prev_{lag}h'] = df['Wind Speed (km/h)'].shift(lag)
        df[f'Temperature_prev_{lag}h'] = df['Temperature (C)'].shift(lag)
        df[f'Humidity_prev_{lag}h'] = df['Humidity'].shift(lag)
        df[f'Pressure_prev_{lag}h'] = df['Pressure (millibars)'].shift(lag)
        df[f'Thunderstorm_prev_{lag}h'] = df['thunderstorm'].shift(lag)

        df[f'Wind_Speed_roll{lag}h'] = df['Wind Speed (km/h)'].shift(1).rolling(lag).mean()
        df[f'Temperature_roll{lag}h'] = df['Temperature (C)'].shift(1).rolling(lag).mean()

    # Cyclical time features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    return df.dropna().reset_index(drop=True)

train_df = add_lag_features(train_df)
val_df = add_lag_features(val_df)
test_df = add_lag_features(test_df)

# ==========================
# 6. Feature Columns
# ==========================
numerical_features = [col for col in train_df.columns 
                      if col not in ['Formatted Date', 'thunderstorm'] + 
                      [f'Wind_Speed_next_{h}h' for h in horizons] +
                      [f'Thunderstorm_next_{h}h' for h in horizons]]

categorical_features = [col for col in train_df.columns if col.startswith("Precip Type_")]

feature_columns = numerical_features + categorical_features

# Ensure numeric type only for columns that exist
for df_ in [train_df, val_df, test_df]:
    cols_to_float = [c for c in feature_columns if c in df_.columns]
    df_[cols_to_float] = df_[cols_to_float].astype(float)

# ==========================
# 7. Evaluation Functions
# ==========================
def evaluate_regression(y_true, y_pred, label="Regression"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n{label} Results:")
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("R²:", r2)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

def evaluate_classification(y_true, y_pred, label="Classification"):
    print(f"\n{label} Results:")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))

# ==========================
# 8. Model Training and Validation per horizon
# ==========================
def train_predict_models(horizon):
    # Prepare targets
    y_wind = train_df[f'Wind_Speed_next_{horizon}h']
    y_thunder = train_df[f'Thunderstorm_next_{horizon}h']

    # --- Wind Speed XGBoost ---
    model_wind = XGBRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1
    )
    model_wind.fit(train_df[feature_columns], y_wind)
    
    # --- Thunderstorm XGBoost with imbalance handling ---
    scale_pos_weight = y_thunder.value_counts()[0] / y_thunder.value_counts()[1]
    model_thunder = XGBClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        random_state=42, n_jobs=-1,
        use_label_encoder=False, eval_metric='logloss'
    )
    model_thunder.fit(train_df[feature_columns], y_thunder)

    # Validation
    X_val = val_df[feature_columns]
    y_val_wind = val_df[f'Wind_Speed_next_{horizon}h']
    y_val_thunder = val_df[f'Thunderstorm_next_{horizon}h']

    pred_wind = model_wind.predict(X_val)
    pred_thunder = model_thunder.predict(X_val)

    print(f"\n=== Horizon: {horizon}h ===")
    evaluate_regression(y_val_wind, pred_wind, label=f"Wind Speed {horizon}h Prediction")
    evaluate_classification(y_val_thunder, pred_thunder, label=f"Thunderstorm {horizon}h Prediction")

# Loop through all horizons
for h in horizons:
    train_predict_models(h)


ValueError: Columns must be same length as key